In [1]:
import requests
import json
import pandas as pd
import psycopg2

In [2]:
# Creando la conexión a Redsshift

url="data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com"
data_base="data-engineer-database"
user="diegoalonsotelloalva1998_coderhouse"
pwd= 'KTs5N49gGd'

try:
    conn = psycopg2.connect(
        host='data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com',
        dbname=data_base,
        user=user,
        password=pwd,
        port='5439'
    )
    print("Conectado a Redshift con éxito!")
    
except Exception as e:
    print("No es posible conectar a Redshift")
    print(e)

Conectado a Redshift con éxito!


In [3]:
#Crear la tabla si no existe
with conn.cursor() as cur:
    cur.execute("""
        CREATE TABLE IF NOT EXISTS diegoalonsotelloalva1998_coderhouse.chistes
        (
        id VARCHAR(255) PRIMARY KEY
        ,categoria VARCHAR(255)   
        ,chiste VARCHAR(255)  
        ,idioma VARCHAR(300)   
        )
    """)
    conn.commit()

In [4]:
#Vaciar la tabla para evitar duplicados o inconsistencias
with conn.cursor() as cur:
  cur.execute("Truncate table chistes")
  count = cur.rowcount

In [5]:
#consultando la tabla
cur = conn.cursor()
cur.execute("SELECT * FROM chistes")
results = cur.fetchall()
results

[]

In [6]:
# La API devuelve un chiste por cada consulta que puede ser de 2 tipos (de 1 parte y 2 partes), en este caso extraeremos
#todos los casos que sean de 1 parte

#Paso 1 Extraer la data

counter = 3
count_init=0
data = []

while count_init <counter: 
    response = requests.get('https://v2.jokeapi.dev/joke/Any')
    data_json = json.loads(response.text)
    
    if data_json['type'] == 'single':
        data.append(data_json)
        count_init= count_init+1
    else:
        count_init=count_init
        
#Paso 2 Crear el diccionario

diccionario = {'Id':[],'Categoria':[],'Chiste':[],'Idioma':[]}

#Extraer cada campo de la data
        
for i in data:
    Id = i['id']
    Categoria=i['category']
    Chiste=i['joke']
    Idioma= i['lang']
    
    diccionario['Id'].append(Id)
    diccionario['Categoria'].append(Categoria)
    diccionario['Chiste'].append(Chiste)
    diccionario['Idioma'].append(Idioma)
    
df = pd.DataFrame(diccionario)
display(df)

,Id,Categoria,Chiste,Idioma
0,82,Pun,I have these weird muscle spasms in my gluteus...,en
1,96,Dark,I didn't vaccinate my 10 kids and the one that...,en
2,72,Misc,Two reasons I don't give money to homeless peo...,en


In [7]:
#Insertando los datos en Redsfhift
from psycopg2.extras import execute_values
with conn.cursor() as cur:
    execute_values(
        cur,
        '''
        INSERT INTO chistes (id,categoria,chiste,idioma)
        VALUES %s
        ''',
        [tuple(row) for row in df.values],
        page_size=len(df)
    )
    conn.commit()

In [8]:
cur.close()
conn.close()